In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import os
import time
from RunningDataset import RunningDataset

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # Encoder
        self.conv1 = nn.Conv2d(in_channels=10, out_channels=64, kernel_size=2, stride=1, padding='same')  # 'same' padding
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=2, stride=1, padding='same')  # 'same' padding
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Bottleneck
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256, 64)  # Adjusted from input (4*4*16 = 256)
        self.fc2 = nn.Linear(64, 5)
        self.fc3 = nn.Linear(5, 64)
        self.fc4 = nn.Linear(64, 256)

        # Decoder
        self.reshape = lambda x: x.view(-1, 16, 4, 4)  # Reshape to (None, 16, 4, 4)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')  # Upsample to (None, 16, 8, 8)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=2, stride=1, padding='same')  # 'same' padding
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=2, stride=1, padding='same')  # 'same' padding
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=10, kernel_size=2, stride=1, padding='same')  # 'same' padding to match input channels
        
    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x
    
    def encode(self, x):
        # Encoder
        #print("Shape input for Layer 1: ", x.shape)
        x = F.selu(self.conv1(x))
        #print("Shape input for Layer 2", x.shape)
        x = F.selu(self.conv2(x))
        #print("Shape input for Layer 3", x.shape)
        x = self.pool(x)
        #print("Shape input for Layer 4", x.shape)
        # Bottleneck
        x = self.flatten(x)
        #print("Shape input for Layer 5", x.shape)
        x = self.fc1(x)
        #print("Shape input for Layer 6", x.shape)
        x = self.fc2(x)
        return x
    
    def decode(self,x):
        #print("Shape input for Layer 7", x.shape)
        x = self.fc3(x)
        #print("Shape input for Layer 8", x.shape)
        x = self.fc4(x)

        # Decoder
        #print("Shape input for Layer 9", x.shape)
        x = self.reshape(x)
        #print("Shape input for Layer 10", x.shape)
        x = self.upsample(x)
        #print("Shape input for Layer 11", x.shape)
        x = F.selu(self.conv3(x))
        #print("Shape input for Layer 12", x.shape)
        x = F.selu(self.conv4(x))
        #print("Shape input for Layer 13", x.shape)
        x = torch.tanh(self.conv5(x))
        #print("Shape output", x.shape)
        return x
    
    def train_model(self, train_loader, num_epochs=100, learning_rate=0.01):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        optimizer = optim.Adadelta(self.parameters(), lr=learning_rate)
        criterion = nn.MSELoss()

        model_dir = 'saved_models'
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        start = time.time()
        
        for epoch in range(num_epochs):
            self.train()
            total_loss = 0
            for data, _ in train_loader:
                data = data.to(device)

                # Forward pass
                outputs = self(data)
                loss = criterion(outputs, data)
                
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()

                optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)

            if (epoch+1) % 10 == 0:
                model_path = os.path.join(model_dir, f'autoencoder_epoch_{epoch+1}.pth')
                torch.save(self.state_dict(), model_path)
                print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

        print('Training completed in {:.3f} seconds.'.format(time.time() - start))
    
    def load_model(self, checkpoint_path):
        self.load_state_dict(torch.load(checkpoint_path))
        self.eval()  # Set the model to evaluation mode
    
    def encode_sample(self, X_sample):
        self.eval()  # Ensure the model is in evaluation mode
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        X_sample = X_sample.to(device)  # Move the sample to the same device as the model
        with torch.no_grad():
            encoded = self.encode(X_sample.unsqueeze(0))  # Add batch dimension
        return encoded

In [3]:
dataset = RunningDataset()
X_train, y_train, X_test, y_test = dataset.preprocess()
train_dataset = TensorDataset(X_train, X_train)  
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
model = Autoencoder()
model.train_model(train_loader)

No injured sample for Athlete ID 11
No injured sample for Athlete ID 12
No injured sample for Athlete ID 15
No injured sample for Athlete ID 25
No injured sample for Athlete ID 30
No injured sample for Athlete ID 31
No injured sample for Athlete ID 46
No injured sample for Athlete ID 60
No injured sample for Athlete ID 73
Original series before clipping:
[0. 1. 0. 1. 0. 0. 0. 0.]
Series after clipping:
[0. 1. 0. 1. 0. 0. 0. 0.]
Original series before clipping:
[0.  0.5 0.5 0.5 0.5 1.  0.5 0.5]
Series after clipping:
[0.  0.5 0.5 0.5 0.5 1.  0.5 0.5]
Original series before clipping:
[0. 0. 0. 1. 0. 0. 0. 0.]
Series after clipping:
[0. 0. 0. 1. 0. 0. 0. 0.]
Original series before clipping:
[0. 0. 0. 0. 0. 0. 1. 0.]
Series after clipping:
[0. 0. 0. 0. 0. 0. 1. 0.]
Original series before clipping:
[0. 0. 1. 0. 0. 0. 0. 1.]
Series after clipping:
[0. 0. 1. 0. 0. 0. 0. 1.]
Original series before clipping:
[0. 0. 0. 0. 0. 1. 0. 1.]
Series after clipping:
[0. 0. 0. 0. 0. 1. 0. 1.]
Original ser

/Users/gabriel/blaze/coaching/coaching_injury_model/venv_coaching_01/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [10/100], Loss: 0.2119
Epoch [20/100], Loss: 0.1172
Epoch [30/100], Loss: 0.1028
Epoch [40/100], Loss: 0.0981
Epoch [50/100], Loss: 0.0950
Epoch [60/100], Loss: 0.0932
Epoch [70/100], Loss: 0.0910
Epoch [80/100], Loss: 0.0894
Epoch [90/100], Loss: 0.0876
Epoch [100/100], Loss: 0.0853
Training completed in 231.650 seconds.


In [6]:
model = Autoencoder()
model.load_model('saved_models/autoencoder_epoch_100.pth')
print(model.encode_sample(X_train[0]))

tensor([[ 2.2906, -2.4428,  0.8046, -0.1086,  1.8486]])
